In [1]:
# import sys
# !{sys.executable} -m pip install osgeo
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
import geopandas as gpd
import glob
import os as os
from os import listdir, getcwd
from os.path import join, abspath
import rasterio
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing
np.set_printoptions(precision=3, suppress=True)
from glob import glob
from os.path import join, abspath
from os import listdir, getcwd
import pandas as pd
from tqdm import tqdm



In [2]:
%%time
RGI = pd.DataFrame()
dir_path = "/home/sa42/data/glac/RGI_TOPO/extracted_outlines/"
full_path = join(abspath(getcwd()), dir_path, "*.shp")
for file in tqdm(glob(full_path)):
    file_left = file[49:]
    file_reg = file_left[:14]
    glac_reader = gpd.read_file(file)
    RGI = RGI.append(glac_reader, ignore_index = True)
    RGI = RGI.drop("BgnDate", axis=1)
    RGI = RGI.drop("EndDate", axis=1)
    RGI = RGI.drop("O1Region", axis=1)
    RGI = RGI.drop("O2Region", axis=1)
    RGI = RGI.drop("Status", axis=1)
    RGI = RGI.drop("Form", axis=1)
    RGI = RGI.drop("TermType", axis=1)
    RGI = RGI.drop("Surging", axis=1)
    RGI = RGI.drop("Linkages", axis=1)
    RGI = RGI.drop("Name", axis=1)
    RGI = RGI.drop("check_geom", axis=1)
    RGI = RGI.drop("Zmed", axis=1)
    RGI = RGI.drop("Connect", axis=1)
    break
# RGI = pd.DataFrame()
# dir_path = "/home/sa42/data/glac/RGI_TOPO/extracted_outlines/"
# full_path = join(abspath(getcwd()), dir_path, "*.shp")
# for file in tqdm(glob(full_path)):
#     file_left = file[49:]
#     file_reg = file_left[:14]
#     sh_reader = gpd.read_file(file)
#     sh_reader["RGI_name"] = os.path.basename(file_reg)
#     RGI = RGI.append(sh_reader, ignore_index = True)



    


  0%|          | 0/216502 [00:00<?, ?it/s]

CPU times: user 1.16 s, sys: 171 ms, total: 1.34 s
Wall time: 1.34 s


In [3]:
import sys
!{sys.executable} -m pip install geos

  Using cached geos-0.2.3-py3-none-any.whl (400 kB)
     |████████████████████████████████| 6.4 MB 29.9 MB/s            
  Using cached Flask-2.0.2-py3-none-any.whl (95 kB)
  Using cached itsdangerous-2.0.1-py3-none-any.whl (18 kB)
  Using cached Jinja2-3.0.3-py3-none-any.whl (133 kB)


In [4]:
RGI

,RGIId,GLIMSId,CenLon,CenLat,Area,Zmin,Zmax,Slope,Aspect,Lmax,geometry
0,RGI60-19.02000,G300617E62419S,-59.3828,-62.4193,7.167,10,270,3.5,51,2827,"POLYGON ((1749.487 -6919708.205, 1725.115 -691..."


In [5]:
import rasterstats as rst
stats = rst.zonal_stats("/home/sa42/data/glac/RGI_TOPO/extracted_outlines/RGI60-19.02000_outlines.shp", 
                    "/home/sa42/data/glac/RGI_TOPO/extracted_dem_files/MAPZEN/RGI60-19.02000_dem.tif",
                       geojson_out=True)
pts = rst.point_query("/home/sa42/data/glac/RGI_TOPO/extracted_outlines/RGI60-19.02000_outlines.shp", 
                  "/home/sa42/data/glac/RGI_TOPO/extracted_dem_files/MAPZEN/RGI60-19.02000_dem.tif")
stats

[{'type': 'Feature',
  'id': '0',
  'properties': OrderedDict([('RGIId', 'RGI60-19.02000'),
               ('GLIMSId', 'G300617E62419S'),
               ('BgnDate', '19570101'),
               ('EndDate', '-9999999'),
               ('CenLon', '-59.3828'),
               ('CenLat', '-62.4193'),
               ('O1Region', '19'),
               ('O2Region', '2'),
               ('Area', '7.167'),
               ('Zmin', '10'),
               ('Zmax', '270'),
               ('Zmed', '-999'),
               ('Slope', '3.5'),
               ('Aspect', '51'),
               ('Lmax', '2827'),
               ('Status', '0'),
               ('Connect', '0'),
               ('Form', '1'),
               ('TermType', '1'),
               ('Surging', '9'),
               ('Linkages', '9'),
               ('Name', None),
               ('check_geom', None),
               ('min', -201.0),
               ('max', -185.0),
               ('mean', -194.50858034321374),
               ('count', 3205)])

In [6]:
import rasterstats as rst
dir_path = "/home/sa42/data/glac/RGI_TOPO/extracted_outlines/"
full_path = join(abspath(getcwd()), dir_path, "*.shp")
sec_path = "/home/sa42/data/glac/RGI_TOPO/extracted_dem_files/MAPZEN/"
sec_full_path = join(abspath(getcwd()), sec_path, "*.tif")


RGI_points = pd.DataFrame()


for shp_file in tqdm(glob(full_path)):
    file_left = shp_file[49:]
    file_reg = file_left[:14]
    dem_file = join(file_reg + "_dem.tif")
    for file in glob(sec_path):
        dem_path = file + dem_file
        pts = rst.point_query(shp_file, dem_path)
        RGI_points = RGI_points.append(pts, ignore_index = True) 
        break
    break

  0%|          | 0/216502 [00:00<?, ?it/s]


In [7]:
len(pts[0])

138

In [8]:
RGI = gpd.GeoDataFrame(RGI)

In [9]:
import rasterstats as rst
stats = rst.zonal_stats("/home/sa42/data/glac/RGI_TOPO/extracted_outlines/RGI60-19.02000_outlines.shp", 
                    "/home/sa42/data/glac/RGI_TOPO/extracted_dem_files/MAPZEN/RGI60-19.02000_dem.tif")
pts = rst.point_query("/home/sa42/data/glac/RGI_TOPO/extracted_outlines/RGI60-19.02000_outlines.shp", 
                  "/home/sa42/data/glac/RGI_TOPO/extracted_dem_files/MAPZEN/RGI60-19.02000_dem.tif")
pts

[[None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  -201,
  -201.0,
  -201.0,
  -201.0,
  -201.0,
  -201.00000000000003,
  -201.0,
  -201.0,
  -201.0,
  -201.0,
  -201.0,
  -201.0,
  -201.0,
  -201.0,
  -201.0,
  -201.0,
  -201.0,
  -200.99999999999997,
  -201.0,
  -201.0,
  -201.00000000000003,
  -201.0,
  -201.0,
  -201.0,
  -201.0,
  -199.73581033787923,
  -196.21341805893462,
  -192.42125576437684,
  -189.18808934016852,
  -189.0,
  -189.0,
  -189.0,
  -188.99999999999997,
  -189.0,
  -189.0,
  -189.0,
  -189.0,
  -189.0,
  -189.0,
  -189.0,
  -189.0,
  -189.00000000000003,
  -188.99999999999997,
  -189.00000000000003,
  -188.6282397343509,
  -187.3889578679195,
  -186.5166503798858,
  -186.0,
  -185.69744530505898,
  -185.1163350430912,
  -185.0,
  -185.0,
  -185,
  None,
  None,
  None,
  None,
  -185,
  -185.0,
  -185,
  None,
  None,
  None,
  None,
  None,
  None,
  -185,
  -185.0,
  -185.0,
  -185.0,
  -185.0,
  -184.99999999999997,
  -185.0,
  -185.0,
 

In [10]:
pts.plot()

AttributeError: 'list' object has no attribute 'plot'

In [ ]:
T = pd.DataFrame(RGI)
T.rename(columns = {"CenLon":"RGI_CenLon",
                     "CenLat":"RGI_CenLat",
                     "Area":"RGI_Area",
                     "Zmin":"RGI_Zmin",
                     "Zmax":"RGI_Zmax",
                     "Slope":"RGI_Slope",
                     "Aspect":"RGI_Aspect",
                     "Lmax":"RGI_Lmax",
                     "geometry":"RGI_geometry",},
          inplace = True)

RGI

In [ ]:
geometryRGI_dem = pd.DataFrame()
dir_path = "/home/sa42/data/glac/RGI_TOPO/extracted_dem_files/MAPZEN/"
full_path = join(abspath(getcwd()), dir_path, "*.tif")
for file in glob(full_path):
    file_left = file[57:]
    file_reg = file_left[:18]
    print(file_reg)
    dem_reader = rasterio.open(file)
    break
#     RGI_dem = RGI_dem.append(dem_reader, ignore_index = True)
#     RGI_dem["RGI_name"] = os.path.basename(file_reg)


In [ ]:
dem_reader = rasterio.open("/home/sa42/data/glac/RGI_TOPO/extracted_dem_files/MAPZEN/RGI60-19.02001_dem.tif")
plt.imshow(dem_reader.read(1))

In [ ]:
%%time
print("System Initiated")
print("Importing glacier area data")
g = gpd.read_file("~/data/glac/glims_download_13173/glims_polygons.shp")
print("import complete")
print("sorting data")
g = g.drop("line_type", axis=1)
g = g.drop("anlys_id", axis=1)
g = g.drop("anlys_time", axis=1)
g = g.drop("primeclass", axis=1)
g = g.drop("src_date", axis=1)
g = g.drop("rec_status", axis=1)
g = g.drop("glac_name", axis=1)
g = g.drop("wgms_id", axis=1)
g = g.drop("local_id", axis=1)
g = g.drop("glac_stat", axis=1)
g = g.drop("subm_id", axis=1)
g = g.drop("release_dt", axis=1)
g = g.drop("proc_desc", axis=1)
g = g.drop("rc_id", axis=1)
g = g.drop("chief_affl", axis=1)
g = g.drop("loc_unc_x", axis=1)CenLon
g = g.drop("loc_unc_y", axis=1)
g = g.drop("glob_unc_x", axis=1)
g = g.drop("glob_unc_y", axis=1)
g = g.drop("submitters", axis=1)
g = g.drop("analysts", axis=1)
g = g.drop("geog_area", axis=1)

glim = gpd.GeoDataFrame(g)
# print("computing areas")
# glim = glim.set_index("glac_id")
# glim["area_calc"] = glim.area

# print("computing boundaries")
# glim["boundary"] = glim.boundary

# print("finding glacier centers")
# glim["point_cent"] = glim.centroid

# glim_cent_first = glim["point_cent"].astype(str).str.split('(').str[1]
# glim_cent = glim_cent_first.astype(str).str.split(')').str[0]
# glim["cent_lat"] = glim_cent.str.split(' ').str[0]
# glim["cent_lon"] = glim_cent.str.split(' ').str[1]
# glim = glim.drop("point_cent", axis=1)CenLon

print("glim complete")

In [ ]:

# T = pd.DataFrame(T_dataset)
# T.rename(columns = {"LAT":"LAT_CENT",
#                     "LON":"LON_CENT"},
#          inplace = True)

# TT = pd.DataFrame(TT_dataset)
# TT.rename(columns = {"LOWER_BOUND":"TT_LOWER_BOUND",
#                      "UPPER_BOUND":"TT_UPPER_BOUND",
#                      "AREA":"TT_AREA",
#                      "MEAN_SLOPE":"TT_MEAN_SLOPE",
#                      "MEAN_THICKNESS":"TT_MEAN_THICKNESS"},
#           inplace = True)

# TTT = pd.DataFrame(TTT_dataset)
# TTT.rename(columns = {"ELEVATION":"POINT_ELEVATION",
#                       "THICKNESS":"POINT_THICKNESS"},
#            inplace = True)

# TTT_plu = pd.merge(TTT, T, how="inner", on=["GlaThiDa_ID"])

# TTT_plus = pd.merge(TT,TTT_plu, how="inner", on=["GlaThiDa_ID"])

# TTT_plus

In [ ]:
T = pd.DataFrame(glim)
T.rename(columns = {"glac_id":"GLIMSId",},
          inplace = True)

In [ ]:
glim

In [ ]:
RGI

In [ ]:
glim.iloc[[1254]].plot()

In [ ]:
testy = glim.drop("area", axis=1)
testy = testy.drop("width", axis=1)
testy = testy.drop("db_area", axis=1)
testy = testy.drop("length", axis=1)
testy = testy.drop("min_elev", axis=1)
testy = testy.drop("max_elev", axis=1)
testy = testy.drop("mean_elev", axis=1)
# testy = testy.drop("area_calc", axis=1)
# testy = testy.drop("boundary", axis=1)
# testy = testy.drop("cent_lat", axis=1)
# testy = testy.drop("cent_lon", axis=1)
# testy = testy.drop("glac_id", axis=1)

testy

In [ ]:
testy_poo = pd.merge(RGI, glim, how="inner", on=["GLIMSId"])

In [ ]:
# testy_poo = testy_poo.dropna()

testy_poo 

In [ ]:
import os
import matplotlib.pyplot as plt
import matplotlib as mpl
import geopandas as gpd
import rioxarray as rxr
from rasterio.plot import plotting_extent
import earthpy as et
import earthpy.spatial as es
import earthpy.plot as ep
fire_boundary_path = "/home/sa42/data/glac/RGI_TOPO/extracted_outlines/RGI60-19.02000_outlines.shp"

In [ ]:
from rasterio.crs import CRS
fire_boundary = gpd.read_file(fire_boundary_path)
fire_boundary.crs

In [ ]:
import rasterio.warp
from rasterio.crs import CRS
from shapely.geometry import box

from pyproj import CRS
from pyproj import Transformer

fire_boundary = Transformer.from_crs(4326, 3857)

xmin, ymin = transformer.transform(-180.0225, -90.0225)
xmax, ymax = transformer.transform(179.9775, 90.0225)
# fire_boundary.crs
fire_boundary

In [ ]:
plt.imshow(fire_boundary.read(1))

In [12]:
import fiona
from shapely.geometry import shape

with fiona.open("/home/sa42/data/glac/RGI_TOPO/extracted_outlines/RGI60-19.02000_outlines.shp", 'r') as features:
    for i, feat in enumerate(features):
        geom = shape(feat['geometry'])
        name = feat['properties']['NAME00']
        print(join((name,) + tuple([str(i) for i in geom.bounds])))

KeyError: 'NAME00'